In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d, UnivariateSpline
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# 1. Data Loading and Exploration

In [2]:
# Load data
train = pd.read_parquet('train_data.parquet')
test = pd.read_parquet('test_data.parquet')
sample_sub = pd.read_csv('sample_submission.csv')

# Display shapes
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Sample submission shape: {sample_sub.shape}")

# Display first few rows of train data
print("\nTrain data head:")
display(train.head())

# Display first few rows of test data
print("\nTest data head:")
display(test.head())

Train shape: (178340, 97)
Test shape: (12065, 96)
Sample submission shape: (12065, 53)

Train data head:


,timestamp,underlying,expiry,call_iv_23500,call_iv_23600,call_iv_23700,call_iv_23800,call_iv_23900,call_iv_24000,call_iv_24100,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
0,1745296089000000000,24160.9,2025-04-24,0.237872,0.213056,0.191247,0.173081,0.157550,0.146430,0.140084,...,0.013995,0.007922,-1.947502e+05,0.024715,0.530894,-0.002354,-3.224848e+05,-1.600795e+06,13063.446970,445511.363636
1,1745304077000000000,24188.1,2025-04-24,0.236015,0.213177,0.189552,0.169672,0.153648,0.141522,0.134405,...,-0.004976,-0.009555,-1.481909e+06,-0.004020,-1.429919,-0.000843,1.658073e+06,-1.742468e+06,31364.628427,-46123.161765
2,1745313495000000000,24148.6,2025-04-24,0.225757,0.199469,0.178547,0.156187,0.140276,0.130288,0.124253,...,-0.012869,-0.004012,-1.250688e+06,-0.035342,-0.523109,0.013778,-2.646675e+06,-5.051008e+07,-847564.971737,-225333.881579
3,1745313499000000000,24147.4,2025-04-24,0.220805,0.195398,0.176247,0.155271,0.139753,0.129641,0.123994,...,-0.006014,0.004207,6.378700e+05,-0.045472,0.741664,0.002590,-1.607321e+06,4.170899e+05,333918.361928,-114960.453869
4,1745313608000000000,24155.9,2025-04-24,0.220088,0.195815,0.177803,0.156409,0.141458,0.130448,0.124707,...,0.020878,-0.001747,9.592702e+04,0.044814,-0.015472,0.012185,6.820360e+05,4.081106e+06,3309.895833,183946.289063



Test data head:


,timestamp,underlying,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
0,0,24735.9,0.280939,NaN,NaN,NaN,0.242149,NaN,0.232439,NaN,...,0.006587,0.002826,-1.737617e+05,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06
1,1,24766.7,0.270276,NaN,0.258893,NaN,NaN,NaN,0.233548,NaN,...,0.005777,0.004588,-3.195086e+05,-0.024106,-0.004696,-0.000158,-4.529075e+06,-1.619022e+06,-0.956928,4.624907e+06
2,2,24896.9,NaN,0.251731,NaN,NaN,0.214869,0.20458,0.194604,NaN,...,0.000829,-0.034944,1.027525e+06,0.028201,0.032234,0.007687,2.663908e+06,0.000000e+00,-0.116264,-2.669766e+06
3,3,24898.1,0.241888,NaN,0.220505,NaN,0.198602,0.18619,NaN,NaN,...,-0.009323,-0.022969,-4.720074e+06,-0.001513,0.008704,-0.000206,-7.672936e+06,-1.903406e+06,-2.249208,-1.858254e+07
4,4,24906.5,0.235328,NaN,0.222983,0.214126,NaN,NaN,0.192603,NaN,...,-0.018674,-0.007588,-4.051681e+05,-0.136267,0.002425,-0.002200,1.334469e+06,2.548789e+05,1.999104,3.024212e+06


# 2. Data Analysis and Preparation

In [3]:
# Get all IV columns from TEST data
iv_columns = [col for col in test.columns if col.startswith(('call_iv_', 'put_iv_'))]

print(f"Number of IV columns: {len(iv_columns)}")
print(f"First 5 IV columns: {iv_columns[:5]}")

# Create strike dictionary from TEST columns
strike_dict = {}
for col in iv_columns:
    strike = col.split('_')[-1]
    if strike not in strike_dict:
        strike_dict[strike] = {'call': None, 'put': None}
    
    if col.startswith('call_iv_'):
        strike_dict[strike]['call'] = col
    else:
        strike_dict[strike]['put'] = col

print("\nStrike dictionary sample:")
print(dict(list(strike_dict.items())[:3]))

Number of IV columns: 52
First 5 IV columns: ['call_iv_24000', 'call_iv_24100', 'call_iv_24200', 'call_iv_24300', 'call_iv_24400']

Strike dictionary sample:
{'24000': {'call': 'call_iv_24000', 'put': 'put_iv_24000'}, '24100': {'call': 'call_iv_24100', 'put': 'put_iv_24100'}, '24200': {'call': 'call_iv_24200', 'put': 'put_iv_24200'}}


# 3. Feature Engineering and Statistics

In [4]:
def calculate_enhanced_stats(data):
    stats = {}
    
    # Calculate per-column statistics
    for col in iv_columns:
        if col in data.columns:
            values = data[col].dropna()
            if len(values) > 0:
                stats[col] = {
                    'mean': values.mean(),
                    'median': values.median(),
                    'std': values.std(),
                    'q25': values.quantile(0.25),
                    'q75': values.quantile(0.75),
                    'skew': values.skew(),
                    'kurt': values.kurtosis()
                }
    
    # Calculate underlying-based statistics
    underlying_stats = {}
    for underlying in data['underlying'].unique():
        subset = data[data['underlying'] == underlying]
        underlying_stats[underlying] = {}
        
        for col in iv_columns:
            if col in subset.columns:
                values = subset[col].dropna()
                if len(values) > 0:
                    underlying_stats[underlying][col] = {
                        'mean': values.mean(),
                        'std': values.std(),
                        'skew': values.skew(),
                        'kurt': values.kurtosis()
                    }
    
    return stats, underlying_stats

global_stats, underlying_stats = calculate_enhanced_stats(train)
overall_mean = np.mean([stats['mean'] for stats in global_stats.values()]) if global_stats else 0.2

print("\nGlobal statistics for first 5 IV columns:")
for col in list(global_stats.keys())[:5]:
    print(f"{col}: {global_stats[col]}")

print("\nUnderlying statistics for first 3 underlyings:")
for underlying in list(underlying_stats.keys())[:3]:
    print(f"Underlying {underlying}: {len(underlying_stats[underlying])} IV columns with stats")


Global statistics for first 5 IV columns:
call_iv_24000: {'mean': np.float64(1.5803173304391611), 'median': np.float64(0.19670149999999997), 'std': np.float64(402.7997710415377), 'q25': np.float64(0.173014), 'q75': np.float64(0.246242), 'skew': np.float64(298.6109287117417), 'kurt': np.float64(89167.4911304059)}
call_iv_24100: {'mean': np.float64(0.1952128715991925), 'median': np.float64(0.17405749999999998), 'std': np.float64(0.05609654334987734), 'q25': np.float64(0.16104975), 'q75': np.float64(0.21034750000000002), 'skew': np.float64(2.328600094123866), 'kurt': np.float64(6.4907873326103225)}
call_iv_24200: {'mean': np.float64(0.16099717636929461), 'median': np.float64(0.1578795), 'std': np.float64(0.032195505401451385), 'q25': np.float64(0.14302949999999998), 'q75': np.float64(0.168074), 'skew': np.float64(2.354673666779333), 'kurt': np.float64(8.370033133706862)}
call_iv_24300: {'mean': np.float64(0.14009934083043626), 'median': np.float64(0.1427675), 'std': np.float64(0.01903019

# 4. Visualization of IV Distributions

In [1]:
# Plot distribution of a sample call and put IV
sample_call = iv_columns[0]
sample_put = iv_columns[1] if len(iv_columns) > 1 else iv_columns[0]

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(train[sample_call].dropna(), kde=True)
plt.title(f'Distribution of {sample_call}')

plt.subplot(1, 2, 2)
sns.histplot(train[sample_put].dropna(), kde=True)
plt.title(f'Distribution of {sample_put}')
plt.tight_layout()
plt.show()

NameError: name 'iv_columns' is not defined

# 5. Enhanced SVI Model Implementation

In [ ]:
def extract_strike_price(strike_str):
    try:
        return float(strike_str)
    except:
        return 0.0

def calculate_moneyness(strike, underlying):
    """Calculate moneyness (strike/underlying)"""
    return strike / underlying if underlying > 0 else 1.0

def enhanced_svi_parametrization(k, a, b, rho, m, sigma, c, d):
    """Enhanced SVI parametrization with additional terms for better smile fitting"""
    return a + b * (rho * (k - m) + np.sqrt((k - m)**2 + sigma**2)) + c * np.exp(-d * (k - m)**2)

def fit_enhanced_svi_smile(strikes, ivs, underlying):
    """Fit enhanced SVI model to volatility smile"""
    if len(strikes) < 4:
        return None
    
    # Convert to log-moneyness
    log_moneyness = np.log(np.array(strikes) / underlying)
    ivs = np.array(ivs)
    
    # Initial parameter guess
    initial_guess = [
        np.mean(ivs),  # a
        0.1,           # b  
        0.0,           # rho
        0.0,           # m
        0.1,           # sigma
        0.01,          # c
        0.1            # d
    ]
    
    def objective(params):
        a, b, rho, m, sigma, c, d = params
        try:
            predicted = enhanced_svi_parametrization(log_moneyness, a, b, rho, m, sigma, c, d)
            return np.sum((ivs - predicted)**2)
        except:
            return 1e6
    
    # Constraints to ensure no-arbitrage
    bounds = [
        (-1, 1),      # a
        (0, 1),       # b
        (-1, 1),      # rho
        (-1, 1),      # m
        (0.01, 1),    # sigma
        (0, 0.1),     # c
        (0, 1)        # d
    ]
    
    try:
        result = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')
        if result.success:
            return result.x
    except:
        pass
    
    return None

# Example SVI fitting visualization
sample_row = train.iloc[0]
underlying = sample_row['underlying']
strikes = []
ivs = []

for col in iv_columns[:10]:  # Just use first 10 for visualization
    if col in sample_row and not pd.isna(sample_row[col]):
        strike = extract_strike_price(col.split('_')[-1])
        strikes.append(strike)
        ivs.append(sample_row[col])

if len(strikes) >= 4:
    svi_params = fit_enhanced_svi_smile(strikes, ivs, underlying)
    
    if svi_params is not None:
        # Create smooth curve for visualization
        test_strikes = np.linspace(min(strikes), max(strikes), 100)
        log_moneyness = np.log(test_strikes / underlying)
        svi_curve = enhanced_svi_parametrization(log_moneyness, *svi_params)
        
        plt.figure(figsize=(10, 6))
        plt.scatter(strikes, ivs, color='red', label='Original Data Points')
        plt.plot(test_strikes, svi_curve, label='Enhanced SVI Fit')
        plt.xlabel('Strike Price')
        plt.ylabel('Implied Volatility')
        plt.title('Enhanced SVI Volatility Smile Fit')
        plt.legend()
        plt.grid()
        plt.show()
    else:
        print("Could not fit SVI to sample data")
else:
    print("Not enough data points for SVI fit visualization")

# 6. Advanced Interpolation Methods

In [ ]:
def advanced_interpolation(strikes, ivs, target_strikes, method='enhanced_svi'):
    """Advanced interpolation methods with enhanced SVI"""
    if len(strikes) < 2:
        return np.full(len(target_strikes), np.mean(ivs) if ivs else 0.2)
    
    # Remove duplicates and sort
    combined = list(zip(strikes, ivs))
    combined = sorted(list(set(combined)))
    strikes, ivs = zip(*combined) if combined else (strikes, ivs)
    
    strikes = np.array(strikes)
    ivs = np.array(ivs)
    target_strikes = np.array(target_strikes)
    
    try:
        if method == 'enhanced_svi':
            # Try enhanced SVI first
            svi_params = fit_enhanced_svi_smile(strikes, ivs, np.mean(strikes))
            if svi_params is not None:
                log_moneyness = np.log(target_strikes / np.mean(strikes))
                predicted = enhanced_svi_parametrization(log_moneyness, *svi_params)
                return np.clip(predicted, 0.01, 0.99)
        
        # Fallback to cubic spline
        if len(strikes) >= 4:
            spline = UnivariateSpline(strikes, ivs, s=len(strikes)*0.01, k=3)
            return np.clip(spline(target_strikes), 0.01, 0.99)
        
        # Fallback to linear interpolation
        f = interp1d(strikes, ivs, kind='linear', bounds_error=False, fill_value='extrapolate')
        return np.clip(f(target_strikes), 0.01, 0.99)
            
    except:
        # Fallback to mean
        return np.full(len(target_strikes), np.mean(ivs))

# Compare interpolation methods
if len(strikes) >= 4:
    test_points = np.linspace(min(strikes), max(strikes), 100)
    
    # Get interpolations
    svi_interp = advanced_interpolation(strikes, ivs, test_points, 'enhanced_svi')
    spline_interp = advanced_interpolation(strikes, ivs, test_points, 'spline')
    linear_interp = advanced_interpolation(strikes, ivs, test_points, 'linear')
    
    plt.figure(figsize=(12, 6))
    plt.scatter(strikes, ivs, color='black', label='Original Data', zorder=5)
    plt.plot(test_points, svi_interp, label='Enhanced SVI')
    plt.plot(test_points, spline_interp, label='Cubic Spline')
    plt.plot(test_points, linear_interp, label='Linear')
    plt.xlabel('Strike Price')
    plt.ylabel('Implied Volatility')
    plt.title('Comparison of Interpolation Methods')
    plt.legend()
    plt.grid()
    plt.show()

# 7. Enhanced Prediction Pipeline

In [ ]:
def enhanced_predict_iv(data):
    data = data.copy()
    
    # Phase 1: Enhanced put-call parity with dynamic adjustment
    for strike, cols in strike_dict.items():
        call_col = cols['call']
        put_col = cols['put']
        
        if call_col in data.columns and put_col in data.columns:
            for idx in data.index:
                call_val = data.at[idx, call_col]
                put_val = data.at[idx, put_col]
                underlying = data.at[idx, 'underlying']
                strike_price = extract_strike_price(strike)
                
                # Calculate theoretical put-call parity adjustment
                moneyness = calculate_moneyness(strike_price, underlying)
                
                if pd.isna(call_val) and not pd.isna(put_val):
                    # Enhanced adjustment based on moneyness and volatility level
                    adjustment = 0.001 * (moneyness - 1.0) * (1 + put_val)
                    data.at[idx, call_col] = max(0.01, put_val + adjustment)
                elif pd.isna(put_val) and not pd.isna(call_val):
                    # Enhanced adjustment based on moneyness and volatility level
                    adjustment = 0.001 * (1.0 - moneyness) * (1 + call_val)
                    data.at[idx, put_col] = max(0.01, call_val + adjustment)
    
    # Phase 2: Enhanced SVI interpolation
    for idx, row in data.iterrows():
        underlying = row['underlying']
        
        # Collect available IV values
        available_data = []
        for col in iv_columns:
            if col in row and not pd.isna(row[col]):
                strike = extract_strike_price(col.split('_')[-1])
                available_data.append((strike, row[col], col))
        
        if len(available_data) >= 2:
            strikes, ivs, cols = zip(*available_data)
            
            # Get missing strikes
            missing_data = []
            for col in iv_columns:
                if col in data.columns and pd.isna(data.at[idx, col]):
                    strike = extract_strike_price(col.split('_')[-1])
                    missing_data.append((strike, col))
            
            if missing_data:
                missing_strikes, missing_cols = zip(*missing_data)
                
                # Use enhanced SVI interpolation
                predicted_ivs = advanced_interpolation(strikes, ivs, missing_strikes, 'enhanced_svi')
                
                # Apply predictions
                for i, col in enumerate(missing_cols):
                    data.at[idx, col] = predicted_ivs[i]
        
        else:
            # Enhanced fallback using underlying price similarity
            current_underlying = row['underlying']
            
            # Find similar underlying prices in training data
            if len(available_data) == 1:
                # Use the single available value as base
                base_iv = available_data[0][1]
            else:
                # Use underlying-specific statistics or global mean
                if current_underlying in underlying_stats:
                    available_cols = [col for col in iv_columns if col in underlying_stats[current_underlying]]
                    if available_cols:
                        base_iv = np.mean([underlying_stats[current_underlying][col]['mean'] for col in available_cols])
                    else:
                        base_iv = overall_mean
                else:
                    base_iv = overall_mean
            
            # Apply base IV to missing values with enhanced noise
            for col in iv_columns:
                if col in data.columns and pd.isna(data.at[idx, col]):
                    # Add small random variation based on column statistics
                    if col in global_stats:
                        noise = np.random.normal(0, global_stats[col]['std'] * 0.01)
                        data.at[idx, col] = max(0.01, base_iv + noise)
                    else:
                        data.at[idx, col] = base_iv
    
    # Phase 3: Cross-sectional consistency enforcement
    for idx, row in data.iterrows():
        strike_iv_pairs = []
        for col in iv_columns:
            if col in data.columns and not pd.isna(data.at[idx, col]):
                strike = extract_strike_price(col.split('_')[-1])
                strike_iv_pairs.append((strike, data.at[idx, col]))
        
        if len(strike_iv_pairs) >= 3:
            strikes, ivs = zip(*strike_iv_pairs)
            strikes = np.array(strikes)
            ivs = np.array(ivs)
            
            # Apply enhanced smoothing to maintain smile shape
            try:
                svi_params = fit_enhanced_svi_smile(strikes, ivs, row['underlying'])
                if svi_params is not None:
                    for col in iv_columns:
                        if col in data.columns:
                            strike = extract_strike_price(col.split('_')[-1])
                            log_moneyness = np.log(strike / row['underlying'])
                            smoothed_iv = enhanced_svi_parametrization(log_moneyness, *svi_params)
                            if 0.01 < smoothed_iv < 1.0:
                                # Blend original and smoothed (70% original, 30% smoothed)
                                data.at[idx, col] = 0.7 * data.at[idx, col] + 0.3 * smoothed_iv
            except:
                pass
    
    return data

# 8. Validation and Model Evaluation

In [ ]:
# Create validation split
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

print(f"Training split size: {len(train_df)}")
print(f"Validation split size: {len(val_df)}")

# Apply to validation set
val_pred = enhanced_predict_iv(val_df)

# Calculate MSE only on originally masked validation points
mse_vals = []
for col in iv_columns:
    if col in val_df.columns and col in val_pred.columns:
        # Focus only on points that were originally missing
        mask = val_df[col].isna() & val_pred[col].notna()
        if mask.any():
            se = (val_df.loc[mask, col] - val_pred.loc[mask, col]) ** 2
            mse_vals.append(se.mean())

validation_mse = np.mean(mse_vals) if mse_vals else 0
print(f"\nValidation MSE (masked points only): {validation_mse:.12f}")

# 9. Test Prediction and Submission

In [ ]:
# Apply to test set
test_pred = enhanced_predict_iv(test)

# Prepare submission
submission = test_pred[['timestamp'] + iv_columns].copy()
submission.columns = sample_sub.columns

# Verify no missing values
assert submission.isna().sum().sum() == 0, "Missing values detected"
submission.to_csv('submission.csv', index=False)

print("\nFinal Submission Preview:")
display(submission.head())
print(f"\nSubmission shape: {submission.shape}")
print(f"Validation MSE: {validation_mse:.12f}")

# 10. Additional Visualizations

In [ ]:
# Visualize predicted IVs for a sample test row
sample_test_row = test_pred.iloc[0]
sample_strikes = []
sample_ivs = []

for col in iv_columns[:20]:  # Just show first 20 for clarity
    if col in sample_test_row:
        strike = extract_strike_price(col.split('_')[-1])
        sample_strikes.append(strike)
        sample_ivs.append(sample_test_row[col])

plt.figure(figsize=(12, 6))
plt.scatter(sample_strikes, sample_ivs)
plt.xlabel('Strike Price')
plt.ylabel('Predicted Implied Volatility')
plt.title('Sample Test Row: Predicted IVs Across Strikes')
plt.grid()
plt.show()